In [138]:
import openai
import csv
from io import StringIO
import time

# Track execeution time
start = time.time()

# Input files
gpt_api_file = "gpt-api.key"
summarized_cv_file = "summarized_cv.txt"
job_offers_csv_file = "job_offers.csv"
output_file = "processed_job_offers.csv"

# Set up the API key
openai.api_key
with open(gpt_api_file, "r") as file:
    # Read the first line of the file
    openai.api_key = file.readline()

# Load Data (CV and job offers)
summarized_cv
with open(summarized_cv_file, "r", encoding='utf-8') as file:
    # Read the first line of the file
    summarized_cv = file.readline()

job_offers
with open(job_offers_csv_file, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    job_offers = list(reader)

# Add columns for structuring results
job_offers[0].append("met")
job_offers[0].append("unmet")
job_offers[0].append("match")
column_names = job_offers[0]

elapsed = time.time()
print("Read all input data: " + "{:.3f}".format(elapsed - start) + " sec")

# Collect ChatGPT Responses for each job_offer
for job in job_offers[1:]:
    
    # Add extra columns to the current job
    job.append("met")
    job.append("unmet")
    job.append("match")
    
    # Generate Prompt
    prompt = "Given my profile and this job offer, list which requirements are met or not. Provide the results in csv format with columns separated by a pipe: requirement|status (met/unmet) \n - my profile: " + summarized_cv + " \n - job offer: " + job[column_names.index('description')]   
    print("Processing: " + job[column_names.index('company')]) # debug info
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                    {"role": "system", "content": "You are a chatbot"},
                    {"role": "user", "content": prompt},
                ],
            timeout=30
        )
       
        result = ''
        for choice in response.choices:
            result += choice.message.content
        # print(result) # debug info

        # Extract result from response and turn into matrix
        parsed_result = StringIO(result)
        result_reader = csv.reader(parsed_result, delimiter='|')
        result_matrix = list(result_reader)

        # print(result_matrix) # debug info
        # Evaluate if result matches the requested structure -> otherwise flag and skip
        result_column_names = result_matrix[0]
        if 'requirement' in result_column_names[0].lower() and 'status' in result_column_names[1].lower(): 
             # Iterate over results
            met_reqs = ""
            unmet_reqs = ""
            for requirement in result_matrix[1:]:
                # collect met/unmet
                try:
                    if requirement[1].lower() == 'met':
                        met_reqs += "- " + requirement[0] + "\n"
                    else:
                        unmet_reqs += "- " + requirement[0] + "\n"
                except IndexError: 
                    # The response does not meet the requested structure
                    print("Error Processing Response: " + str(requirement))

            # calculate metric - check that at least one req was processed
            total_reqs = len(met_reqs) + len(unmet_reqs)
            if total_reqs > 0:
                match = len(met_reqs) / total_reqs * 100
            else:
                match = 'Error Processing Response'

            # append results to row
            job[column_names.index('met')] = met_reqs
            job[column_names.index('unmet')] = unmet_reqs
            job[column_names.index('match')] = match
        else:
            print("Error while reading ChatGPT response to job offer: " + str(job))
            print("Response: " + str(result_matrix))
            job.append("met")
            job.append("unmet")
            job.append("match")
            job[column_names.index('met')] = "Error: could not process this job offer"
            job[column_names.index('unmet')] = "Error: could not process this job offer"
            job[column_names.index('match')] = "Error: could not process this job offer"
    except requests.exceptions.Timeout:
        print("Request timed out")
        job[column_names.index('met')] = "Error: Requested Timed out"
        job[column_names.index('unmet')] = "Error: Requested Timed out"
        job[column_names.index('match')] = "Error: Requested Timed out"

elapsed = time.time()
print("Done Processing: " + str(len(job_offers)-1) + " - Time: " + "{:.3f}".format(elapsed - start) + " sec")

    
# Write output file
# Debug output
#for row in job_offers:
#    print(str(row[0]) + " " + str(row[1]) + " " + str(row[4]) + " " + str(row[5]) + " " + str(row[6]))
with open(output_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the data to the CSV file
    writer.writerows(job_offers)

print("Output file: " + output_file)
elapsed = time.time()
print("Done! - Total Time: " + "{:.3f}".format(elapsed - start) + " sec")


Read all input data: 0.002 sec
Processing: ByteWorks
Processing: CodeWave
Processing: LogicLabs
Processing: MindMeld
Processing: PixelPulse
Processing: TechTonic
Processing: SynapseSoft
Processing: ByteBuilders
Processing: CyberCore
Processing: DataDreams
Processing: Tessitura
Processing: Bits of Stock
Processing: Pluto Health
Processing: Leadstack Inc
Processing: Avalon Healthcare Solutions
Processing: Pragma Platform
Processing: ConsultingLLC
Processing: AIWorks
Processing: TechMatters
Processing: All Infra
Done Processing: 21 - Time: 199.602 sec
Output file: processed_job_offers.csv
Done! - Total Time: 199.605 sec
